In [59]:
import cv2

In [60]:
def name_map(x):
    if x=='sachin':
        return 0
    if x=='obama':
        return 1
    if x=='messi':
        return 2

In [61]:
def get_name(x):
    if x==0:
        return 'sachin'
    if x==1:
        return 'obama'
    if x==2:
        return 'messi'

In [62]:
#ROI - region of interest
# we need to identify just the face and no need of any background ie ROI
# 1.convert the image to gray scale
# 2.use the preprogrammed xml to identify the faces
# 3.pass the image to preprogrammed cascade using detectMultiScale
# 4.If there are multiple people in the picture we are selecting the first one only
# 5.Crop the image top of head left(x) to top of head right(x+w) & bottom of head left(y) to top of head left(y+h)
#This makes a square patter w mean width and h means height x and y are the cordinates in the graph
def detect_face(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_cascade=cv2.CascadeClassifier(r'C:\Users\febia\OneDrive\Desktop\Document\Others\Mariam\Learning\MachineLearning\DL Projects\face recognition\haarcascade_frontalface_default.xml')
    face=face_cascade.detectMultiScale(gray,minNeighbors=4)
    #Crop the image with the x,y,w,h values
    if len(face)==0:
        return None,None#if no value is returned it will create an error so this helps not to return any error
    x,y,w,h=face[0]
    return gray[x:x+w,y:y+h],face[0]

In [63]:
import os
faces=[]
labels=[]

In [64]:
#Try yo fetch the training pictures from the sub folders
def prepare_training_data(data_folder):
    dirs=os.listdir(data_folder)
    # print(dirs)

    for dir_names in dirs:
        sub_dir=data_folder+"/"+dir_names
        # print(sub_dir)
        images=os.listdir(sub_dir)
        # print(images)
        for img in images:
            imgpath=sub_dir+"/"+img
            # print(imgpath)
            image=cv2.imread(imgpath)
            cv2.imshow('training images',image)
            cv2.waitKey(500)
            
#For each picture we get ge the cropped image to faces list and append the corresponding labels
            #detect face
            face,rect=detect_face(image)
            if face is not None:
                faces.append(face)
                labels.append(name_map(dir_names))
                # print(labels)
    cv2.waitKey(1)            
    cv2.destroyAllWindows()
    return faces,labels

In [65]:
faces,labels=prepare_training_data(r'C:\Users\febia\OneDrive\Desktop\Document\Others\Mariam\Learning\MachineLearning\DL Projects\face recognition\DB')
print(labels)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [66]:
len(labels)#actually 30 
#only 29 came as Sachines one face not detected

29

In [67]:
#  pip install --user opencv-contrib-python
# https://pyimagesearch.com/2021/05/03/face-recognition-with-local-binary-patterns-lbps-and-opencv/#The%20Local%20Binary%20Patterns%20(LBPS)%20For%20Face%20Recognition%20Algorithm
# Once a face has been detected in an image, the first step is to divide the face ROI into 7×7 equally sized cells.
# Then, for each of these cells, we compute a Local Binary Pattern histogram.
# This spatial encoding also allows us to weigh the resulting histograms from each of the cells differently
# , giving more discriminative power to more distinguishing features of the face:
# Finally, the weighted 7×7 LBP histograms are concatenated together to form the final feature vector.

In [68]:
# Performing face recognition is done using the x^2 distance and a nearest neighbor classifier:

# 1.A face is presented to the system
# 2.LBPs are extracted, weighted, and concatenated in the same manner as the training data
# 3.k-NN (with k=1) is performed with the x^2 distance to find the closest face in the training data.
# 4.The name of the person associated with the face with the smallest x^2 distance is chosen as the final classification

In [69]:
recognizer=cv2.face.LBPHFaceRecognizer_create()#Local Binary Pattern Histogram
recognizer

< cv2.face.LBPHFaceRecognizer 000001706AC505B0>

In [70]:
#We are trying to train the faces with LBPHFaceRecognizer
import numpy as np
recognizer.train(faces,np.array(labels))

In [71]:
def draw_rect(img,rect):
    (x,y,w,h)=rect
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,0),2)

In [72]:
def write_txt(img,text,x,y):
    cv2.putText(img,text,(x,y),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)

In [73]:
#We are trying to predict a face
# 1.input the faces
# 2.LBPH used to predict
# 3.Get the name to write on the predicted Image
# 4.Draw a rectangle in the ROI
def predict_new(test_img):
    face,rect=detect_face(test_img)
    label=recognizer.predict(face)
    label_text=get_name(label[0])
    write_txt(test_img,label_text,rect[0],rect[1])
    draw_rect(test_img,rect)
    return test_img

In [74]:
test_img=cv2.imread(r'C:\Users\febia\OneDrive\Desktop\Document\Others\Mariam\Learning\MachineLearning\DL Projects\face recognition\WhatsApp Image 2024-01-02 at 18.54.17_8f7fa609.jpg')
predicted = predict_new(test_img)
cv2.imshow("IMAGE",predicted)
cv2.waitKey()
cv2.destroyAllWindows()

KeyboardInterrupt: 